In [41]:
import sys
print(sys.executable)


c:\Users\Gur Levy\heinecan-4\.venv\Scripts\python.exe


In [42]:
!pip install requests


In [43]:
import requests
import csv
from datetime import datetime, timedelta


In [44]:
# api endpoint and params
url = 'https://api.openweathermap.org/data/3.0/onecall'
params = {
    'lat': 25.686613,
    'lon': -100.316116,
    'exclude': 'current,minutely,daily,alerts',
    'appid': 'd420319993e5906012bee411669261e0',
    'units': 'metrics'
}
#api request
response = requests.get(url, params = params)
data = response.json()
#timezone offset
offset = data.get('timezone_offset', 0)
#csv file
csv_rows = []
header = ['datetime', 'temp',  'feels_like', 'pressure', 'humidity', 'dew_point', 'uvi', 'clouds','wind_speed', 'wind_deg', 'weather_description']
#changing to datetime
for entry in data['hourly']:
    dt = datetime.utcfromtimestamp(entry['dt']) + timedelta(seconds=offset)
    weather_desc = entry['weather'][0]['description'] if entry.get('weather')else''
    row = [
        dt.strftime("%Y-%m-%d %H:%M:%S"),
        entry.get("temp"),
        entry.get("feels_like"),
        entry.get("pressure"),
        entry.get("humidity"),
        entry.get("dew_point"),
        entry.get("uvi"),
        entry.get("clouds"),
        entry.get("wind_speed"),
        entry.get("wind_deg"),
        weather_desc
    ]
    csv_rows.append(row)

#csv save
csv_file = 'monterrey_hourly_weather.csv'
with open(csv_file, 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(csv_rows)

print(f'file saved{csv_file}')


file savedmonterrey_hourly_weather.csv


C:\Users\Gur Levy\AppData\Local\Temp\ipykernel_11516\2351409520.py:20: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  dt = datetime.utcfromtimestamp(entry['dt']) + timedelta(seconds=offset)


In [45]:
import requests
import csv
from datetime import datetime, timedelta

# monterrey location and api 
lat = 25.686613
lon = -100.316116
api_key = 'd420319993e5906012bee411669261e0'

# hourly weather from One Call API
weather_url = 'https://api.openweathermap.org/data/3.0/onecall'
weather_params = {
    'lat': lat,
    'lon': lon,
    'exclude': 'current,minutely,daily,alerts',
    'appid': api_key,
    'units': 'metric' 
}

weather_response = requests.get(weather_url, params=weather_params)
weather_data = weather_response.json()
offset = weather_data.get('timezone_offset', 0)

# build dict by datetime string (hour) for easy merging
weather_by_hour = {}
for entry in weather_data['hourly']:
    dt = datetime.utcfromtimestamp(entry['dt']) + timedelta(seconds=offset)
    dt_str = dt.strftime("%Y-%m-%d %H")
    weather_by_hour[dt_str] = {
        'datetime': dt.strftime("%Y-%m-%d %H:%M:%S"),
        'temp': entry.get("temp"),
        'humidity': entry.get("humidity"),
        'wind_speed': entry.get("wind_speed")
    }

# hourly solar radiation from Solar API 
solar_url = 'https://api.openweathermap.org/energy/1.0/solar/data'
solar_by_hour = {}

for i in range(7):
    date_str = (datetime.now() + timedelta(days=i)).strftime('%Y-%m-%d')
    params = {
        'lat': lat,
        'lon': lon,
        'date': date_str,
        'appid': api_key
    }

    response = requests.get(solar_url, params=params)
    if response.status_code == 200:
        data = response.json()
        hourly = data['irradiance']['hourly']
        for h in hourly:
            dt_str = f"{date_str} {h['hour']:02}"
            solar_by_hour[dt_str] = h['clear_sky']['ghi']  # W/m²
    else:
        print(f"Solar API failed for {date_str}: {response.status_code}")

# merge and create final csv
final_rows = []
header = ['datetime', 'temp', 'humidity', 'wind_speed', 'solar_ghi']

for dt_hour, weather in weather_by_hour.items():
    solar = solar_by_hour.get(dt_hour)
    if solar is not None:
        row = [
            weather['datetime'],
            weather['temp'],
            weather['humidity'],
            weather['wind_speed'],
            solar
        ]
        final_rows.append(row)

# save file
csv_file = 'monterrey_utci_inputs.csv'
with open(csv_file, 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(final_rows)

print(f'UTCI input file saved: {csv_file}')


C:\Users\Gur Levy\AppData\Local\Temp\ipykernel_11516\303758452.py:27: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  dt = datetime.utcfromtimestamp(entry['dt']) + timedelta(seconds=offset)


UTCI input file saved: monterrey_utci_inputs.csv


In [46]:
!pip install pandas

In [47]:
import pandas as pd
import numpy as np
# merged data
df = pd.read_csv('monterrey_utci_inputs.csv')

# estimate MRT (mean radiant temperature)
# MRT = temp + 0.03 * GHI 
df['mrt'] = df['temp'] + 0.03 * df['solar_ghi']

# calculate water vapor pressure (pa) using Tetens formula
es = 6.112 * np.exp((17.67 * df['temp']) / (df['temp'] + 243.5))  # hPa
df['pa'] = (es * (df['humidity'] / 100)) / 10  # convert to kPa

# this is NOT the official UTCI model, empirical regression
# Bröde et al. (2012) — simplified terms only
df['utci'] = (
    0.6076 +
    0.9772 * df['temp'] +
    -2.2584 * df['wind_speed'] +
    0.3984 * (df['mrt'] - df['temp']) +
    5.1273 * df['pa']
)


df.to_csv('monterrey_utci_results.csv', index=False)

print("UTCI calculation complete. File saved: monterrey_utci_results.csv")


UTCI calculation complete. File saved: monterrey_utci_results.csv


In [48]:
df.head()

,datetime,temp,humidity,wind_speed,solar_ghi,mrt,pa,utci
0,2025-05-24 03:00:00,24.34,85,0.57,0.00,24.3400,2.588074,36.375191
1,2025-05-24 04:00:00,24.77,84,0.22,0.00,24.7700,2.624337,37.771758
2,2025-05-24 05:00:00,24.31,83,0.31,0.00,24.3100,2.522634,36.597531
3,2025-05-24 06:00:00,23.97,82,1.07,83.81,26.4843,2.441859,35.136438
4,2025-05-24 07:00:00,24.76,81,1.19,294.50,33.5950,2.529098,38.602884


In [49]:
## utci_scores, sperate working hours (8-18), and calculate avrg, dropdown of graphs

In [50]:
# utci score function
def classify_utci(utci_value):
    if utci_value <= 26:
        return "1"
    elif utci_value <= 32:
        return "2"
    elif utci_value <= 38:
        return "3"
    elif utci_value <= 46:
        return "4"
    else:
        return "5"
    
df['heat_risk_score'] = df['utci'].apply(classify_utci)
df.to_csv("monterrey_utci_with_measures.csv", index=False)

df.head()


,datetime,temp,humidity,wind_speed,solar_ghi,mrt,pa,utci,heat_risk_score
0,2025-05-24 03:00:00,24.34,85,0.57,0.00,24.3400,2.588074,36.375191,3
1,2025-05-24 04:00:00,24.77,84,0.22,0.00,24.7700,2.624337,37.771758,3
2,2025-05-24 05:00:00,24.31,83,0.31,0.00,24.3100,2.522634,36.597531,3
3,2025-05-24 06:00:00,23.97,82,1.07,83.81,26.4843,2.441859,35.136438,3
4,2025-05-24 07:00:00,24.76,81,1.19,294.50,33.5950,2.529098,38.602884,4


In [51]:
#function day and night shift
df['datetime'] = pd.to_datetime(df['datetime'])

def shift(dt):
    hour = dt.hour
    return 1 if 8 <= hour < 18 else 0 
df['shift'] = df['datetime'].apply(shift)
df.to_csv("monterrey_utci_with_measures.csv", index=False)


df.head()

,datetime,temp,humidity,wind_speed,solar_ghi,mrt,pa,utci,heat_risk_score,shift
0,2025-05-24 03:00:00,24.34,85,0.57,0.00,24.3400,2.588074,36.375191,3,0
1,2025-05-24 04:00:00,24.77,84,0.22,0.00,24.7700,2.624337,37.771758,3,0
2,2025-05-24 05:00:00,24.31,83,0.31,0.00,24.3100,2.522634,36.597531,3,0
3,2025-05-24 06:00:00,23.97,82,1.07,83.81,26.4843,2.441859,35.136438,3,0
4,2025-05-24 07:00:00,24.76,81,1.19,294.50,33.5950,2.529098,38.602884,4,0


In [65]:
# merge datasets for measures
df['heat_risk_score'] = df['heat_risk_score'].astype(int)

df_measures = pd.read_csv("heat_risk_database.csv")
merged_df = pd.merge(df, df_measures,left_on='heat_risk_score', right_on='Heat_Risk_Score',how='left')

merged_df.to_csv("monterrey_utci_with_measures.csv", index=False)


In [63]:
merged_df.head()

,datetime,temp,humidity,wind_speed,solar_ghi,mrt,pa,utci,heat_risk_score,shift,Heat_Risk_Score,Measure 1,Measure 2,Measure 3
0,2025-05-24 03:00:00,24.34,85,0.57,0.00,24.3400,2.588074,36.375191,3,0,3,smth,smth,smth
1,2025-05-24 04:00:00,24.77,84,0.22,0.00,24.7700,2.624337,37.771758,3,0,3,smth,smth,smth
2,2025-05-24 05:00:00,24.31,83,0.31,0.00,24.3100,2.522634,36.597531,3,0,3,smth,smth,smth
3,2025-05-24 06:00:00,23.97,82,1.07,83.81,26.4843,2.441859,35.136438,3,0,3,smth,smth,smth
4,2025-05-24 07:00:00,24.76,81,1.19,294.50,33.5950,2.529098,38.602884,4,0,4,smth,smth,smth
